In [54]:
from rdflib import Graph
import sys
sys.path.insert(0, '..')
from utils import query_to_df
import csv
import os
from pathlib import Path

In [55]:
gs = Graph(store = 'Oxigraph')
gs.parse('s223-example.ttl', format='ttl')
gb = Graph(store = 'Oxigraph')
gb.parse('brick-example-parsed.ttl')


<Graph identifier=N5f1374aec80e46f9bab91dea00e7ea63 (<class 'rdflib.graph.Graph'>)>

In [66]:
def read_queries_from_csv(csv_path):
    """
    Read queries from a CSV file.
    
    Args:
        csv_path: Path to the CSV file
        
    Returns:
        List of dictionaries containing query information
    """
    queries = []
    
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            query_info = {
                'query_id': row.get('query_id', ''),
                'question': row.get('question', ''),
                'generated_sparql': row.get('generated_sparql', ''),
                'model': row.get('model', ''),
                'source_file': os.path.basename(csv_path)
            }
            queries.append(query_info)
    
    return queries

# Directory containing the CSV files
results_dir = Path('react_kgqa_results')

# Find all CSV files
csv_files = list(results_dir.glob('*.csv'))

# print(f"Found {len(csv_files)} CSV files in {results_dir}\n")

all_queries = []

# Process each CSV file
for csv_file in csv_files:
    # print(f"Processing: {csv_file.name}")
    queries = read_queries_from_csv(csv_file)
    all_queries.extend(queries)
    # print(f"  - Found {len(queries)} queries\n")
    g_i = gs if '_s223' in csv_file.name else gb
    for query in all_queries:
        print(csv_file.name)
        print(query)
        print(query['question'])
        try:
            display(query_to_df(query['generated_sparql'], g_i))
        except Exception as e:
            print(e)

ReAct(w100)_s223.csv
{'query_id': 'custom_q_1', 'question': "I'm too cold, find the components most likely at fault.", 'generated_sparql': 'PREFIX s223: <http://data.ashrae.org/standard223#>\nPREFIX qudt: <http://qudt.org/schema/qudt/>\nPREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\n\nSELECT DISTINCT ?component ?label WHERE {\n  # Restrict to physical components (devices, equipment, points, or s223:Component)\n  ?component a ?type .\n  FILTER(\n    ?type IN (\n      s223:Component,\n      s223:QuantifiableObservableProperty,\n      s223:QuantifiableActuatableProperty\n    )\n  )\n  # Look for components involved in temperature control\n  OPTIONAL { ?component rdfs:label ?label }\n  FILTER (\n    (BOUND(?label) && (\n      CONTAINS(LCASE(STR(?label)), "damper") ||\n      CONTAINS(LCASE(STR(?label)), "reheat") ||\n      CONTAINS(LCASE(STR(?label)), "coil") ||\n   

,component,label
0,urn:example#vav-with-reheat_name-ztemp_3_9,QuantifiableObservableProperty
1,urn:example#vav-with-reheat_name-ztemp_3_8,QuantifiableObservableProperty
2,urn:example#vav-with-reheat_name-ztemp_3_7,QuantifiableObservableProperty
3,urn:example#vav-with-reheat_name-ztemp_3_6,QuantifiableObservableProperty
4,urn:example#vav-with-reheat_name-ztemp_3_5,QuantifiableObservableProperty
...,...,...
251,urn:example#multiple-zone-ahu_oa_temp_0,QuantifiableObservableProperty
252,urn:example#multiple-zone-ahu_ma_temp_3,QuantifiableObservableProperty
253,urn:example#multiple-zone-ahu_ma_temp_2,QuantifiableObservableProperty
254,urn:example#multiple-zone-ahu_ma_temp_1,QuantifiableObservableProperty


ReAct(w100)_s223.csv
{'query_id': 'custom_q_1', 'question': 'What are all the instances of valve commands for cooling only VAVs?', 'generated_sparql': 'PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX s223: <http://data.ashrae.org/standard223#>\n\nSELECT ?vav ?vav_label ?valve_command ?valve_command_label WHERE {\n  # Find all VAVs (assuming a generic VAV class)\n  ?vav rdf:type s223:VAV .\n  OPTIONAL { ?vav rdfs:label ?vav_label }\n  \n  # Filter for cooling-only VAVs via label or property (if available)\n  FILTER(\n    (BOUND(?vav_label) && CONTAINS(LCASE(?vav_label), "cooling-only"))\n    # Add additional FILTERs here if a property or tag for cooling-only exists, e.g.:\n    # || EXISTS { ?vav s223:hasCoolingOnly true }\n  )\n  \n  # Find associated valve command points via possible relationships\n  {\n    ?vav s223:hasPoint ?valve_command .\n  } UNION {\n    ?vav s223:hasValveCommand ?valve_command .\n  } UNION {

,vav,vav_label,valve_command,valve_command_label


ReAct(w100)_s223.csv
{'query_id': 'custom_q_2', 'question': 'What are all the components of each air handling unit?', 'generated_sparql': 'PREFIX s223: <http://data.ashrae.org/standard223#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX : <urn:example#>\n\nSELECT ?ahu ?ahu_label ?component ?component_label WHERE {\n  # Identify AHUs by class or label\n  {\n    ?ahu a s223:AirHandlingUnit .\n  } UNION {\n    ?ahu rdfs:label ?ahu_label_raw .\n    FILTER(CONTAINS(LCASE(STR(?ahu_label_raw)), "air handling unit"))\n  }\n  # Find components via likely relationships\n  OPTIONAL { ?ahu s223:hasComponent ?component . }\n  OPTIONAL { ?ahu :hasComponent ?component . }\n  OPTIONAL { ?ahu :hasPart ?component . }\n  # Only keep actual components (not classes or properties)\n  ?component a ?component_type .\n  FILTER(\n    ?component_type != rdf:Property &&\n    ?component_type != rdfs:Class &&\n    !STRSTARTS(STR(?component_ty

...

,valve_command


ReAct(w5000)_s223.csv
{'query_id': 'custom_q_2', 'question': 'What are all the components of each air handling unit?', 'generated_sparql': 'PREFIX s223: <http://data.ashrae.org/standard223#>\nPREFIX : <urn:example#>\n\nSELECT ?ahu ?ahu_label ?component ?component_label\nWHERE {\n  ?ahu a s223:AirHandlingUnit .\n  OPTIONAL { ?ahu rdfs:label ?ahu_label }\n  ?ahu s223:contains ?component .\n  OPTIONAL { ?component rdfs:label ?component_label }\n}\nORDER BY ?ahu ?component\n', 'model': 'openai/gpt-4.1', 'source_file': 'ReAct(w5000)_s223.csv'}
What are all the components of each air handling unit?


,ahu,ahu_label,component,component_label
0,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_clg_coil_0,Coil
1,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_htg_coil_0,Coil
2,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_ra_damper_0,Damper
3,urn:example#multiple-zone-ahu_name_0,AirHandlingUnit,urn:example#multiple-zone-ahu_sa_fan_0,Fan
4,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_clg_coil_1,Coil
5,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_htg_coil_1,Coil
6,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_ra_damper_1,Damper
7,urn:example#multiple-zone-ahu_name_1,AirHandlingUnit,urn:example#multiple-zone-ahu_sa_fan_1,Fan
8,urn:example#multiple-zone-ahu_name_2,AirHandlingUnit,urn:example#multiple-zone-ahu_clg_coil_2,Coil
9,urn:example#multiple-zone-ahu_name_2,AirHandlingUnit,urn:example#multiple-zone-ahu_htg_coil_2,Coil


ReAct(w5000)_s223.csv
{'query_id': 'custom_q_3', 'question': 'What are the discharge air temperature sensors for all VAVs and their associated AHUs?', 'generated_sparql': 'PREFIX s223: <http://data.ashrae.org/standard223#>\nPREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>\n\nSELECT DISTINCT ?vav ?dat_sensor ?ahu WHERE {\n  # Find all VAVs (SingleDuctTerminal)\n  ?vav a s223:SingleDuctTerminal .\n\n  # Find discharge air temperature sensors for the VAV\n  {\n    ?vav s223:hasProperty ?dat_sensor .\n  } UNION {\n    ?vav s223:contains ?dat_sensor .\n  }\n  ?dat_sensor s223:hasQuantityKind quantitykind:Temperature .\n  # Accept either hasAspect or hasRole for discharge\n  OPTIONAL { ?dat_sensor s223:hasAspect ?aspect . }\n  OPTIONAL { ?dat_sensor s223:hasRole ?aspect . }\n  FILTER(\n    COALESCE(STR(?aspect),"") = "http://data.ashrae.org/standard223#Role-Discharge"\n    || COALESCE(STR(?aspect),"") = "Role-Discharge"\n    || COALESCE(STR(?aspect),"") = "Discharge"\n    || COALES

,vav,dat_sensor,ahu


In [ ]:
# running fourth s223 Query
query = """PREFIX ns1: <http://data.ashrae.org/standard223#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT DISTINCT ?component ?componentType ?role
WHERE {
  # Find all air handling units and zone terminals
  ?system a ?systemType .
  FILTER (?systemType IN (ns1:AirHandlingUnit, ns1:SingleDuctTerminal))
  # Get their contained components
  ?system ns1:contains ?component .
  # Identify component type and role
  ?component a ?componentType .
  OPTIONAL { ?component ns1:hasAspect ?role . }
  # Components likely at fault for "too cold":
  FILTER (
    ?componentType IN (ns1:Coil, ns1:Damper, ns1:Fan)
    ||
    EXISTS { ?component ns1:hasAspect ns1:Role-Cooling }
    ||
    EXISTS { ?component ns1:hasAspect ns1:Role-Heating }
  )
}
ORDER BY ?system ?component"""

In [4]:
query_to_df(query,gs)

,component,componentType,role
0,urn:example#multiple-zone-ahu_clg_coil_0,s223:Coil,s223:Role-Cooling
1,urn:example#multiple-zone-ahu_htg_coil_0,s223:Coil,s223:Role-Heating
2,urn:example#multiple-zone-ahu_ra_damper_0,s223:Damper,None
3,urn:example#multiple-zone-ahu_sa_fan_0,s223:Fan,s223:Role-SupplyAir
4,urn:example#multiple-zone-ahu_clg_coil_1,s223:Coil,s223:Role-Cooling
...,...,...,...
191,urn:example#vav-with-reheat_rhc_3_7,s223:Coil,s223:Role-Heating
192,urn:example#vav-with-reheat_dmp_3_8,s223:Damper,None
193,urn:example#vav-with-reheat_rhc_3_8,s223:Coil,s223:Role-Heating
194,urn:example#vav-with-reheat_dmp_3_9,s223:Damper,None


In [35]:
gb = Graph(store = 'Oxigraph')
gb.parse('brick-example-parsed.ttl')
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?component ?component_type ?label ?point ?point_type
WHERE {
  # Find components that are Heating_Coil, Supply_Fan, or Damper
  ?component a ?component_type ;
             rdfs:label ?label .
  FILTER(?component_type IN (
      brick:Heating_Coil,
      brick:Supply_Fan,
      brick:Damper
  ))

  # Optionally get their command/status points
  OPTIONAL {
    ?component brick:hasPoint ?point .
    ?point a ?point_type .
    FILTER(?point_type IN (
      brick:Valve_Position_Command,
      brick:Position_Command,
      brick:Start_Stop_Command,
      brick:Fan_Status,
      brick:Damper_Position_Command
    ))
  }
}
ORDER BY ?component"""

In [36]:
query_to_df(query, gb)

,component,component_type,label,point,point_type
0,urn:example#multiple-zone-ahu_htg_coil_0,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
1,urn:example#multiple-zone-ahu_htg_coil_1,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
2,urn:example#multiple-zone-ahu_htg_coil_2,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
3,urn:example#multiple-zone-ahu_htg_coil_3,brick:Heating_Coil,Heating_Coil,urn:example#multiple-zone-ahu_htg_coil-valve_c...,brick:Valve_Position_Command
4,urn:example#multiple-zone-ahu_ra_damper_0,brick:Damper,Damper,urn:example#multiple-zone-ahu_ra_damper-dmppos_0,brick:Damper_Position_Command
...,...,...,...,...,...
191,urn:example#vav-with-reheat_rhc_3_5,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_5,brick:Position_Command
192,urn:example#vav-with-reheat_rhc_3_6,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_6,brick:Position_Command
193,urn:example#vav-with-reheat_rhc_3_7,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_7,brick:Position_Command
194,urn:example#vav-with-reheat_rhc_3_8,brick:Heating_Coil,Heating_Coil,urn:example#vav-with-reheat_rhc-cmd_3_8,brick:Position_Command


In [53]:
# Running react s223 queries

In [37]:
# preparing parsed 223 example 


In [38]:
s223 = Graph()
# s223.parse('s223-example.ttl')
s223.parse('../../../223standard/models/core.ttl')
s223.parse('../../../223standard/models/equipment.ttl')
s223.parse('../../../223standard/vocab/enumeration.ttl')
# s223.serialize('s223-example-parsed.ttl', format = 'ttl')
s223.serialize('s223-relevant-ontology.ttl', format = 'ttl')

<Graph identifier=N0fc82a55470740c1bbf9f4ebd9ca57c9 (<class 'rdflib.graph.Graph'>)>

In [39]:
#  Brick React 100 queries 
# NO QUERIES RETURNED RESULTS

In [ ]:
brick = Graph(store = 'Oxigraph')
brick.parse('../ontologies/Brick14.ttl', format = 'ttl')
## Error 
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?vav ?valve_command WHERE {
  ?vav a ?vav_type .
  ?vav_type rdfs:subClassOf* brick:VAV .
  ?vav brick:hasPoint ?valve_command .
  ?valve_command a ?cmd_type .
  ?cmd_type rdfs:subClassOf* brick:Valve_Command .
  # Identify cooling-only VAVs by class or property if available
  # Example: ?vav brick:hasMode brick:Cooling_Only .
}"""
query_to_df(query, gb)

,vav,valve_command


In [ ]:
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX : <urn:example#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?ahu ?component WHERE {
  ?ahu a brick:Air_Handling_Unit .
  ?ahu (brick:hasPart|brick:hasSubComponent)+ ?component .
}"""
query_to_df(query, gb)

,ahu,component


In [ ]:
query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?vav ?vav_name ?sensor ?sensor_name ?ahu ?ahu_name WHERE {
  # Find all VAVs
  ?vav a brick:VAV .
  OPTIONAL { ?vav rdfs:label ?vav_name }
  # Find discharge air temperature sensors for each VAV
  ?vav brick:hasPoint ?sensor .
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  OPTIONAL { ?sensor rdfs:label ?sensor_name }
  # Find associated AHU for each VAV
  ?vav brick:isFedBy ?ahu .
  ?ahu a brick:AHU .
  OPTIONAL { ?ahu rdfs:label ?ahu_name }
}"""
query_to_df(query, gb)

,vav,vav_name,sensor,sensor_name,ahu,ahu_name


In [ ]:
# Brick React 5000 queries

In [ ]:
# Fails because it's looking for the valve command on the vav, not on the valve of the coil 
query = """
PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?vav ?valve_cmd WHERE {
  ?vav a brick:VAV .
  FILTER NOT EXISTS {
    ?vav brick:hasPart ?heating_coil .
    ?heating_coil a brick:Heating_Coil .
  }
  ?vav brick:hasPoint ?valve_cmd .
  ?valve_cmd a brick:Position_Command .
}"""
query_to_df(query, gb)

,vav,valve_cmd


In [ ]:
# Fails because fo Air Handling Unit alias 
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
SELECT ?ahu ?component WHERE {
  ?ahu a brick:Air_Handling_Unit .
  ?ahu brick:hasPart+ ?component .
}"""
query_to_df(query, gb)

,ahu,component


In [ ]:
# SUCCESS! Also for more specific query 
query = """SELECT ?vav ?ahu ?sensor WHERE {
  ?vav a brick:VAV .
  ?vav brick:hasPoint ?sensor .
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  ?ahu brick:feeds ?vav .
  ?ahu a ?ahu_type .
  FILTER(?ahu_type IN (brick:AHU, brick:Air_Handling_Unit))
}"""
query_to_df(query, gb)

,vav,ahu,sensor
0,urn:example#vav-with-reheat_name_3_9,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_9
1,urn:example#vav-with-reheat_name_3_8,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_8
2,urn:example#vav-with-reheat_name_3_7,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_7
3,urn:example#vav-with-reheat_name_3_6,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_6
4,urn:example#vav-with-reheat_name_3_5,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name-dat_3_5
...,...,...,...
115,urn:example#vav-cooling-only_name_0_12,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_12
116,urn:example#vav-cooling-only_name_0_11,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_11
117,urn:example#vav-cooling-only_name_0_10,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_10
118,urn:example#vav-cooling-only_name_0_1,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_dat_0_1


In [25]:
# updaing DA KGQA for Brick using openai 4.1

In [ ]:
queries = ["""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?vav ?vavLabel ?sensor ?sensorLabel ?ahu ?ahuLabel
WHERE {
  ?vav a brick:VAV .
  OPTIONAL { ?vav rdfs:label ?vavLabel }
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  OPTIONAL { ?sensor rdfs:label ?sensorLabel }
  ?sensor brick:isPointOf ?vav .
  OPTIONAL {
    ?vav (brick:feeds|^brick:feeds|brick:isFedBy|^brick:isFedBy)+ ?ahu .
    ?ahu a brick:AHU .
    OPTIONAL { ?ahu rdfs:label ?ahuLabel }
  }
}""",
"""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?ahu ?ahuLabel ?component ?componentLabel WHERE {
  ?ahu a brick:Air_Handling_Unit ;
       rdfs:label ?ahuLabel .
  ?ahu ?rel ?component .
  FILTER(?rel IN (brick:Relationship, brick:hasAssociatedTag))
  OPTIONAL { ?component rdfs:label ?componentLabel }
}""",
"""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?valveCommand
WHERE {
  ?valveCommand rdf:type brick:Valve_Command .
}""",
"""PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?component ?label ?faultTag WHERE {
  {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Cold .
  } UNION {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Box .
  }
  OPTIONAL { ?component rdfs:label ?label }
  OPTIONAL {
    ?faultTag a brick:Tag ;
      brick:isAssociatedWith ?component .
    FILTER regex(str(?faultTag), "Fault", "i")
  }
}"""]

for query in queries:
    print(query)
    display(query_to_df(query, gb))

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?vav ?vavLabel ?sensor ?sensorLabel ?ahu ?ahuLabel
WHERE {
  ?vav a brick:VAV .
  OPTIONAL { ?vav rdfs:label ?vavLabel }
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  OPTIONAL { ?sensor rdfs:label ?sensorLabel }
  ?sensor brick:isPointOf ?vav .
  OPTIONAL {
    ?vav (brick:feeds|^brick:feeds|brick:isFedBy|^brick:isFedBy)+ ?ahu .
    ?ahu a brick:AHU .
    OPTIONAL { ?ahu rdfs:label ?ahuLabel }
  }
}


,vav,vavLabel,sensor,sensorLabel,ahu,ahuLabel


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?ahu ?ahuLabel ?component ?componentLabel WHERE {
  ?ahu a brick:Air_Handling_Unit ;
       rdfs:label ?ahuLabel .
  ?ahu ?rel ?component .
  FILTER(?rel IN (brick:Relationship, brick:hasAssociatedTag))
  OPTIONAL { ?component rdfs:label ?componentLabel }
}


,ahu,ahuLabel,component,componentLabel


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?valveCommand
WHERE {
  ?valveCommand rdf:type brick:Valve_Command .
}


,valveCommand


PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?component ?label ?faultTag WHERE {
  {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Cold .
  } UNION {
    ?component (brick:Relationship|brick:hasAssociatedTag) tag:Box .
  }
  OPTIONAL { ?component rdfs:label ?label }
  OPTIONAL {
    ?faultTag a brick:Tag ;
      brick:isAssociatedWith ?component .
    FILTER regex(str(?faultTag), "Fault", "i")
  }
}


,component,label,faultTag
0,brick:Cold_Deck,Cold Deck,None
1,brick:Cold_Water_Storage_Tank,Cold Water Storage Tank,None
2,brick:Cold_Box,Cold Box,None
3,brick:Variable_Air_Volume_Box,Variable Air Volume Box,None
4,brick:Variable_Air_Volume_Box_With_Reheat,Variable Air Volume Box With Reheat,None
5,brick:Constant_Air_Volume_Box,Constant Air Volume Box,None
6,brick:Hot_Box,Hot Box,None
7,brick:Hospitality_Box,Hospitality Box,None
8,brick:Environment_Box,Environment Box,None
9,brick:Box_Mode_Command,Box Mode Command,None


In [49]:
import pyshacl
gb_rdflib = Graph()
gb_rdflib.parse('brick-example.ttl')
brick = Graph()
brick.parse('../ontologies/Brick144.ttl')
v = pyshacl.validate(data_graph = gb_rdflib, shacl_graph=gb_rdflib + brick, ont_graph=brick, inplace=True, inference = 'owlrl')

In [ ]:
# looks like its just because I'm not running reasoning on the model (adding inverse relations)
# I can run reasoning first or just add inverse relatoins maybe 
# Works on reasoned version
query = """PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT *
WHERE {
  ?vav a brick:VAV .
  ?sensor a brick:Discharge_Air_Temperature_Sensor .
  ?sensor brick:isPointOf ?vav .
  ?vav (brick:feeds|^brick:feeds|brick:isFedBy|^brick:isFedBy)+ ?ahu .
  ?ahu a brick:AHU .
}"""
query_to_df(query, gb_rdflib)

,sensor,ahu,vav
0,urn:example#vav-cooling-only_dat_0_0,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_0
1,urn:example#vav-cooling-only_dat_0_1,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_1
2,urn:example#vav-cooling-only_dat_0_10,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_10
3,urn:example#vav-cooling-only_dat_0_11,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_11
4,urn:example#vav-cooling-only_dat_0_12,urn:example#multiple-zone-ahu_name_0,urn:example#vav-cooling-only_name_0_12
...,...,...,...
115,urn:example#vav-with-reheat_name-dat_3_5,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_5
116,urn:example#vav-with-reheat_name-dat_3_6,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_6
117,urn:example#vav-with-reheat_name-dat_3_7,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_7
118,urn:example#vav-with-reheat_name-dat_3_8,urn:example#multiple-zone-ahu_name_3,urn:example#vav-with-reheat_name_3_8
